<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js101_astroObjectsD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js101_astroObjectsD2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.07 created,  last updated on 2024.12.25
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js101_astroObjectsD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js101_astroObjectsD2D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.05.05 created, last updated on 2018.12.07
//    ver 0.0.1  2019.01.27 v1, last updated on 2021.09.23
//    ver 0.0.2  2021.11.07 v2, last updated on 2021.11.07
//    ver 0.0.3  2023.05.12 v3, last updated on 2023.09.10
//
// --------------------  gravity  - classical dynamics 2D
//
//  astro objects - dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    force: F(r) = -G*mi*mj/r^2                     (r>r0) gravity
//                = -20.0*G*mi*mj*(r-0.95*r0)/r0^3   (r<=r0) spring
//                     spring: F(r) = -k(r-0.95*r0),
//                     continuty: F(r0) = -G*mi*mj/r0^2  --> k = 20*G*mi*mj/r0^3
//
// --------------------
*/

const astroObjectsD2D = (function(){ // ====================  astroObjectsD2D Module  ====================

	const g_gg = 6.67408e-11;			// (m^3 kg^-1 s^-2) gravitational constant
	const g_AA = 1.0e5;					// (m) length unit g_AA = 100 (km)
	const g_nMax = 1000;				// array max

	let g_Nmt = 1000;					// number of ao
	let g_sysTime = 0.0;				// (s) system time
	let g_timeStep = 10.0;				// (s) time step dt
	let g_potentialEnergy = 0.0;		// (J) total potential energy
	let g_kineticEnergy = 0.0;			// (J) total kinrtic energy

	const g_xx = dim1( g_nMax );		// (m) position x-component
	const g_yy = dim1( g_nMax );		// (m) position y-component
	const g_vx = dim1( g_nMax );		// (m/s) velocity x-component
	const g_vy = dim1( g_nMax );		// (m/s) velocity y-component
	const g_ffx = dim1( g_nMax );		// (N) x-component of total force
	const g_ffy = dim1( g_nMax );		// (N) y-component of total force
	const g_mas = dim1( g_nMax );		// (kg) mass of astro object
	const g_rc =  dim1( g_nMax );		// (m) radius of astro object
	const g_damp = dim1( g_nMax );		// daming force between i-th and j*th object
										// = -0.5*(g_damp[i]+g_damp[j])*vij^2

	function dim1( n ) {
		return new Float64Array( n );
	}


	// --------------------  astro objects data  --------------------

	const aObject = [
		// 0:mas(kg) 1:rc(m) 2:damp 3:hue color
		[  1.0e18, 0.8*g_AA, 1.0e8, 0.85 ],
		[  1.0e18, 0.8*g_AA, 5.0e8, 0.01 ],
		[  5.0e18, 0.8*g_AA, 2.0e8, 0.10 ],
		[ 10.0e18, 0.8*g_AA, 5.0e8, 0.20 ] ];


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		g_sysTime = 0.0;
		setAOs( theme );
	}

	function setAOs(theme) {
		const knd = 1;
		const rcOfKind = aObject[knd][1];

		if (theme==0) { // 2-body
			let i = 0;
			//setAOdisc(ii, knd, aa, x0, y0, r0, th, vx0, vy0)
			i = setAOdisc(i,knd, 0.6*rcOfKind,-15.0*g_AA, 0.0 *g_AA, rcOfKind*3.0, 0.0,   0.0,  0.0);
			i = setAOdisc(i,knd, 0.6*rcOfKind, 30.0*g_AA, 0.0 *g_AA, rcOfKind*2.0, 0.0,   0.0, 20.0);
			g_Nmt = i;
		} else if (theme==1) { // 3-body
			let i = 0;
			i = setAOdisc(i,knd, 0.6*rcOfKind,-15.0*g_AA,  0.0 *g_AA, rcOfKind*3.0, 0.0,   0.0,  0.0);
			i = setAOdisc(i,knd, 0.6*rcOfKind, 30.0*g_AA,  0.0 *g_AA, rcOfKind*1.5, 0.0,   0.0, 30.0);
			i = setAOdisc(i,knd, 0.6*rcOfKind,-10.0*g_AA,-20.0 *g_AA, rcOfKind*1.5, 0.0, -50.0,  0.0);
			g_Nmt = i;
		} else if (theme==2) { // disk
			let i = 0;
			i = setAOdisc(i,knd, 4.0*rcOfKind, 0.0*g_AA,  0.0*g_AA, rcOfKind*30.0, 0.0,   0.0,  0.0);
			g_Nmt = i;
			const v0 = 50.0;
			for (let ii=0; ii<g_Nmt; ii++) {
				const r = Math.sqrt(g_xx[ii]*g_xx[ii]+g_yy[ii]*g_yy[ii]);
				g_vx[ii] = -v0*g_yy[ii]/(rcOfKind*30);
				g_vy[ii] = v0*g_xx[ii]/(rcOfKind*30);
			}

		}

		removeTranslationalMotion();
	}

	function setAOdisc(ii, knd, aa, x0, y0, r0, th, vx0, vy0) {
		let ip = ii;
		const a = aa;
		const b = (Math.sqrt(3.0)/2.0)*a;
		const nx = Math.floor(2.0*r0/b) + 1;
		const ny = Math.floor(2.0*r0/a) + 1;
		const sth = Math.sin(th);
		const cth = Math.cos(th);

		for (let i=0; i<nx; i++) {
			const x = b*i;
			for (let j=0; j<ny; j++) {
				let y = a*j; if (i%2==1) y += 0.5*a;
				if ((x-r0)*(x-r0)+(y-r0)*(y-r0)<=r0*r0) {
					const xp = x-r0, yp = y-r0;
					g_xx[ip] = x0 + cth*xp - sth*yp;
					g_yy[ip] = y0 + sth*xp + cth*yp;
					g_vx[ip] = vx0;
					g_vy[ip] = vy0;
					g_ffy[ip] = 0.0;
					g_ffy[ip] = 0.0;
					g_mas[ip] = aObject[knd][0];
					g_rc[ip] = aObject[knd][1];
					g_damp[ip] = aObject[knd][2];
					ip += 1;
				}
			}
		}
		return ip;
	}

	function removeTranslationalMotion() {
		const nmt=g_Nmt;

		let m=0.0, mvx=0.0, mvy=0.0;
		for (let i=0; i<nmt; i++) {
			m += g_mas[i];
			mvx += g_mas[i]*g_vx[i];
			mvy += g_mas[i]*g_vy[i];
		}
		const vtx = mvx/m, vty = mvy/m;
		for (let i=0; i<nmt; i++) {
			g_vx[i] -= vtx;
			g_vy[i] -= vty;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( nCalc ) {

		for (let i=0; i<nCalc; i++) {
			g_sysTime += g_timeStep;
			moveAOs(g_timeStep);
		}
	}

	function moveAOs(dt) {
		const nmt=g_Nmt;

		for (let i=0; i<nmt; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nmt; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}
		g_kineticEnergy = totalKineticEnergy();
	}

	function calcForce() {
		const nmt=g_Nmt;

		for (let i=0; i<nmt; i++) {
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
		}
		g_potentialEnergy = 0.0;

		for (let i=0; i<nmt; i++) {
			for (let j=i+1; j<nmt; j++) {
				const xij = g_xx[i]-g_xx[j], yij = g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				const f = force(rij,i,j);
				g_potentialEnergy += -g_gg*g_mas[i]*g_mas[j]/rij;
				const fxij = f*xij/rij, fyij = f*yij/rij;
				g_ffx[i] +=  fxij;
				g_ffy[i] +=  fyij;
				g_ffx[j] += -fxij;
				g_ffy[j] += -fyij;
			}
		}
	}

	function force(r, i, j) {
		const r0 = 0.5*(g_rc[i]+g_rc[j]);
		const gmm = g_gg*g_mas[i]*g_mas[j];

		let f;
		if (r>r0) {
			f = -gmm/(r*r);
		} else {
			f = -20.0*gmm*(r-0.95*r0)/(r0*r0*r0);
			f += -0.5*(g_damp[i]+g_damp[j])*((g_xx[i]-g_xx[j])*(g_vx[i]-g_vx[j])
							+(g_yy[i]-g_yy[j])*(g_vy[i]-g_vy[j]));
		}
		return f;
	}


	// --- utility

	function totalKineticEnergy() {
		const nmt=g_Nmt;

		let s=0.0;
		for (let i=0; i<nmt; i++) {
			s += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return s;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme )
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		getSysParam:	function() { return [ g_Nmt, 100.0*g_AA, g_timeStep ]; },
		getSysTime:		function() { return g_sysTime; },
		getRc:			function(i) { return g_rc[i]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
	};

})(); // ====================  astroObjectsD2D end  ====================


const js101 = (function(){ // ====================  js Module  ====================

	const theModule = astroObjectsD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;		// 0:2-body  1:3-body  2:disk
	let v_nCalc = 20;

	let p_Nmt, p_xLen, p_timeStep;	// <-- theModule.getSysParam();

  let sysTime;
	let nowData = [];
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];

	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

  let breakFlag = false;
	let perticleFlag = true;


	// mouse wheel for zoom in/out
	const g2d = {};
	g2d.zoom = 1.0;

	g2d.mouseWheel = function(e) {
		g2d.deltaY = e.deltaY;
		if ( g2d.deltaY > 0 ) g2d.zoom *= 0.95;
		else if ( g2d.deltaY < 0 ) g2d.zoom *= 1.05;
		if ( g2d.zoom<0.25 ) g2d.zoom = 0.25;
		if ( g2d.zoom>2.0 ) g2d.zoom = 2.0;
	};


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;

			canvas.addEventListener("mousewheel", g2d.mouseWheel);
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme );
			[ p_Nmt, p_xLen, p_timeStep ] = theModule.getSysParam();

      perticleFlag = true;
      xxList = [];
      yyList = [];
      vxList = [];
      vyList = [];
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
		}

		draw( ctx, dispMode );

    if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		nowData = [ sysTime, p_Nmt ]
    for (let i=0; i<p_Nmt; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const boxSize = 400;
		const cx0 = xCanvasSize/2, cy0 = yCanvasSize/2-20, sc = boxSize/p_xLen*g2d.zoom;


		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// draw box
		const bSize = boxSize*g2d.zoom;
		ctx.strokeStyle = "#444444";
		ctx.strokeRect( cx0-bSize/2, cy0-bSize/2, bSize, bSize );

		if ( dispMode==0 ) {
			drawAOs( ctx, 0, cx0, cy0, sc );
		} else if ( dispMode==1 ) {
			drawAOs( ctx, 1, cx0, cy0, sc );
		}

		// caption
		sysTime = theModule.getSysTime();
		ctx.fillStyle = "#888888";
		ctx.fillText( `box = 10000 x 10000 (km)`, 20, yCanvasSize-30 );
		ctx.fillText( `N = "${p_Nmt}`, 20, yCanvasSize-10 );
		ctx.fillText( `time = ${sysTime.toFixed(1)} (s) = ${(sysTime/3600/24).toFixed(3)}(days)`, 100, yCanvasSize-10 );
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"

	}

	function drawAOs( ctx, drawMode, cx0, cy0, sc ) {
		const nn = p_Nmt, twoPi = 2.0*Math.PI, ts = 2000*p_timeStep;

		ctx.fillStyle = "rgb(180, 80, 0)";
		for (let i=0; i<nn; i++) {
			let x, y;
			[ x, y ] = theModule.getPosition(i);
			const rc = theModule.getRc(i);
			let r = rc*sc*0.5;
			if (r<1.0) r = 1;
			ctx.beginPath();
			ctx.arc( x*sc + cx0, -y*sc + cy0, r, 0, twoPi, false);
			ctx.fill();
			if ( drawMode==1 ) {
				let vx, vy;
				[ vx, vy ] = theModule.getVelocity(i);
				const x2 = x + vx*ts, y2 = y + vy*ts;
				drawLine( ctx, x*sc + cx0, -y*sc + cy0, x2*sc + cx0, -y2*sc + cy0, "#aa8866" );
			}
		}
	}

	function drawLine(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g2d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, p_Nmt ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()

		setDispMode:	setDispMode,	// setDispMode()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, p_Nmt ]
    pygetParticlesList: pygetParticlesList, //() :return [ nowData, xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js101 module end  ====================


const js = js101;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js101] Astro Objects - dynamics under gravity 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>2-body</option><option>3-body</option><option>disk</option>
</select>
    <span style="margin-right: 200px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>ball + velocity</option>
</select>
<br>

mouse-wheel to zoom in/out
    <span style="margin-right: 20px;"></span>
<button onClick="js.viewHome()">return to initial zoom</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<a href="javascript:history.back()">back</a> | <a href="http://mike1336.web.fc2.com/index.html">top</a>
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get system data and print
for i in range(10):
  [ sysTime, nn ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  system time = {sysTime:>10.1f} (s),  N = {nn:>3d}')
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme, dispMode

import time

themeList = [ '0: 2-body', '1: 3-body', '2; disk' ]
dispModeList = [ '0: ball', '1: ball + velocity' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change theme and dispMode
for theme in [ 0, 1, 2 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 1 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "-- -- dispMode:", dispModeList[dispMode], "--" )
    time.sleep(5)
    # get data and display
    [ sysTime, nn ] = eval_js( 'js.pygetData({})'.format(i) )
    print(f'\t  system time = {sysTime:>10.1f} (s),  N = {nn:>3d}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get particles data and save

import time
import numpy as np

themeList = [ '0: 2-body', '1: 3-body', '2; disk' ]
dispModeList = [ '0: ball', '1: ball + velocity' ]

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 1 # '1: 3-body'
eval_js( 'js.pysetTheme({})'.format(theme) )
print(f"-- theme: {themeList[theme]} --")

# wait
wait_time = 10
print(f'-- wait {wait_time} s --')
time.sleep(wait_time)

# get particle data
print("-- get astro-object data --")
[ nowData, xxList, yyList, vxList, vyList ] = eval_js('js.pygetParticlesList()')
[ sysTime, nn ] = nowData
print(f'got system time = {sysTime:>10.1f} (s),  N = {nn:>3d}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# pack astro-object data
packed_data = np.array([ xxList, yyList, vxList, vyList ])

# save packed data
file_name = 'js101_data.npy'
print(f'-- save paked data as {file_name} --')
np.save( file_name, packed_data )

# convert to nampy array
print("-- convert to numpy array Dens --")
X = np.array(xxList)   # x-position of every astro-object
Y = np.array(yyList)   # y-position of every astro-object
Vx =  np.array(vxList) # x-velocity of every astro-object
Vy =  np.array(vyList) # y-velocity of every astro-object
print(f"X shape :{X.shape}, Y shape :{Y.shape}, Vx shape :{Vx.shape} , Vy shape :{Vy.shape}  ")

In [ ]:
# load particles data

import numpy as np

# load packed data
file_name = 'js101_data.npy'
print(f'-- load paked data from {file_name} --')
packed_data = np.load( file_name )

# unpack data and set numpy data for plot
X = packed_data[0]   # x-position of every astro-object
Y = packed_data[1]   # y-position of every astro-object
Vx =  packed_data[2] # x-velocity of every astro-object
Vy =  packed_data[3] # y-velocity of every astro-object
print(f"X shape :{X.shape}, Y shape :{Y.shape}, Vx shape :{Vx.shape} , Vy shape :{Vy.shape}  ")

In [ ]:
# scatter plot Astro-Object

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.xlim(-5e6, 5e6)
plt.ylim(-5e6, 5e6)
plt.title(f"astro object (10000km x 10000km)  N={len(X)}")
plt.scatter(X, Y)
plt.show()

In [ ]:
# velosity space

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.xlabel("Vx(m/s)")
plt.ylabel("Vy(m/s)")
plt.xlim(-100, 100)
plt.ylim(-100, 100)
plt.title(f"velocity soace  N={len(Vx)}")
plt.scatter(Vx, Vy)
plt.show()

In [ ]:
# histogram of velosity

V = np.sqrt(Vx**2 + Vy**2)

plt.figure(figsize=(8, 8))
plt.xlabel("V(m/s)")
plt.xlim(0, 50)
plt.title(f"histogram of velocity  N={len(Vx)}")
plt.hist(V,30)
plt.show()

In [ ]:
# correlation matrix X,Y,Vx,Vy
# colab AI wrote:
# prompt: show correlation matrix X,Y,Vx,Vy

import pandas as pd

print(f'-- correration matrix N={len(X)}, 1/sqrt(N)={1/np.sqrt(len(X)):10.6f} --')

# Create a DataFrame from the data
df = pd.DataFrame({'X': X, 'Y': Y, 'Vx': Vx, 'Vy': Vy})

# Calculate the correlation matrix
corr = df.corr()

# Display the correlation matrix
print(corr)

In [ ]:
# prompt: in astro-objects, categorize objects that have close objects in the same class. ( every object in the class  must have the same category )
# Not working as expected.( expected clasify in 0,1,2 (3-body)) , Every near object must have the the same category

import pandas as pd

# Assuming X, Y, Vx, Vy are already defined from the previous code

# Create a DataFrame
df = pd.DataFrame({'X': X, 'Y': Y, 'Vx': Vx, 'Vy': Vy})

# Define a distance threshold (adjust as needed)
distance_threshold = 1e5  # Example: 100,000 meters

# Function to categorize objects
def categorize_objects(df, distance_threshold):
    categories = {}
    next_category_id = 0
    categorized_objects = []

    for i in range(len(df)):
      if i in categorized_objects:
        continue

      current_category = []
      current_category.append(i)
      categorized_objects.append(i)

      for j in range(i+1, len(df)):
          if j not in categorized_objects:
              distance = np.sqrt((df['X'][i] - df['X'][j])**2 + (df['Y'][i] - df['Y'][j])**2)
              if distance < distance_threshold:
                  current_category.append(j)
                  categorized_objects.append(j)
      if current_category:
        categories[next_category_id] = current_category
        next_category_id += 1

    return categories


# Categorize objects
categories = categorize_objects(df, distance_threshold)

# Assign categories to the DataFrame
df['category'] = -1  # Initialize with -1 (unassigned)
for category_id, object_indices in categories.items():
  for index in object_indices:
    df.loc[index, 'category'] = category_id

# Print the DataFrame with categories
df

In [ ]:
# Gemini wrote:

import pandas as pd

# Assuming X, Y, Vx, Vy are already defined from the previous code

# Create a DataFrame
df = pd.DataFrame({'X': X, 'Y': Y, 'Vx': Vx, 'Vy': Vy})

# Define a distance threshold (adjust as needed)
distance_threshold = 2e5 # chanhge 1e5 -->2e5 1e5  # Example: 100,000 meters

def categorize_objects(df, distance_threshold):
    categories = {}
    next_category_id = 0
    categorized_objects = []

    for i in range(len(df)):
        if i not in categorized_objects:
            categories[next_category_id] = [i]  # Start a new category
            categorized_objects.append(i)
            next_category_id += 1  # Increment for the next potential category

    # Merge categories based on distance
    for i in range(len(categories)):
        for j in range(i + 1, len(categories)):
            # Check distance between any two objects in the categories
            for obj1 in categories[i]:
                for obj2 in categories[j]:
                    distance = np.sqrt((df['X'][obj1] - df['X'][obj2])**2 + (df['Y'][obj1] - df['Y'][obj2])**2)
                    if distance < distance_threshold:
                        # Merge categories if distance is below the threshold
                        categories[i].extend(categories[j])
                        categories[j] = []  # Empty the merged category
                        break  # Break inner loop if a close object is found
                else:
                    continue  # Continue if the inner loop didn't break
                break  # Break outer loop if the inner loop broke

    # Remove empty categories and re-index
    categories = {k: v for k, v in categories.items() if v}
    categories = {i: objects for i, objects in enumerate(categories.values())}

    return categories


# Categorize objects
categories = categorize_objects(df, distance_threshold)

# Assign categories to the DataFrame
df['category'] = -1  # Initialize with -1 (unassigned)
for category_id, object_indices in categories.items():
  for index in object_indices:
    df.loc[index, 'category'] = category_id

# Print the DataFrame with categories
df

In [ ]:
# scatter plot of astro objects

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.xlim(-5e6, 5e6)
plt.ylim(-5e6, 5e6)
plt.title(f"astro object (10000km x 10000km)  N={len(X)}")
plt.scatter(X, Y)
plt.show()

In [ ]:
# prompt: colord by categories in 'scatter plot of astro objects'

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# ... (your existing code for loading and preprocessing data) ...

# Assuming df is your DataFrame with 'X', 'Y', and 'category' columns

plt.figure(figsize=(8, 8))
plt.xlabel("x(m)")
plt.ylabel("y(m)")
plt.xlim(-5e6, 5e6)
plt.ylim(-5e6, 5e6)
plt.title(f"astro object (10000km x 10000km)  N={len(X)}")

# Iterate through categories and plot each with a different color
for category in df['category'].unique():
    subset = df[df['category'] == category]
    plt.scatter(subset['X'], subset['Y'], label=f'Category {category}')

plt.legend()  # Show the legend
plt.show()